In [1]:
%run 10_ngram_common.ipynb

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
# !pip install -U datasets
# !pip install fsspec==2023.9.2
# pattern = "no_keep"
# pattern = "keep_all"
pattern = "new"

In [9]:
# Read the dataset
data_df = pd.read_csv('torgo_new.csv')
dataset_csv = load_dataset('csv', data_files='torgo_new.csv',cache_dir=None)

speakers = data_df['speaker_id'].unique()

print(f'Speakers: {", ".join(speakers)}')

Speakers: F01, F03, F04, FC01, FC02, FC03, M01, M02, M03, M04, M05, MC01, MC02, MC03, MC04


In [10]:
atypical_speaker_texts = {}

for speaker_id in atypical_speakers:
    test_speaker=speaker_id
    valid_speaker = 'F03' if test_speaker != 'F03' else 'F04'
    train_speaker = [s for s in speakers if s not in [test_speaker, valid_speaker]]
    
    torgo_dataset = DatasetDict()
    torgo_dataset['train'] = dataset_csv['train'].filter(lambda x: x in train_speaker, input_columns=['speaker_id'])
    torgo_dataset['validation'] = dataset_csv['train'].filter(lambda x: x == valid_speaker, input_columns=['speaker_id'])
    torgo_dataset['test'] = dataset_csv['train'].filter(lambda x: x == test_speaker, input_columns=['speaker_id'])

    # print(torgo_dataset)
    if pattern == "no_keep":
        unique_texts = set(torgo_dataset['train'].unique(column='text')) | set(torgo_dataset['validation'].unique(column='text')) | set(torgo_dataset['test'].unique(column='text'))
        unique_texts_count = {}
        
        for text in unique_texts:
          unique_texts_count[text] = {'train_validation': 0, 'test': 0}
        
        for text in torgo_dataset['train']['text']:
          unique_texts_count[text]['train_validation'] += 1
        
        for text in torgo_dataset['validation']['text']:
          unique_texts_count[text]['train_validation'] += 1
        
        for text in torgo_dataset['test']['text']:
          unique_texts_count[text]['test'] += 1
        
        texts_to_keep_in_train_validation = []
        texts_to_keep_in_test = []
        for text in unique_texts_count:
          if unique_texts_count[text]['train_validation'] < text_count_threshold and unique_texts_count[text]['test'] > 0:
            texts_to_keep_in_test.append(text)
          else:
            texts_to_keep_in_train_validation.append(text)
        
        original_data_count = {'train': len(torgo_dataset['train']), 'validation': len(torgo_dataset['validation']), 'test': len(torgo_dataset['test'])}
        
        # Update the three dataset splits
        torgo_dataset['train'] = torgo_dataset['train'].filter(lambda x: x['text'] in texts_to_keep_in_train_validation)
        torgo_dataset['validation'] = torgo_dataset['validation'].filter(lambda x: x['text'] in texts_to_keep_in_train_validation)
        torgo_dataset['test'] = torgo_dataset['test'].filter(lambda x: x['text'] in texts_to_keep_in_test)
        
        print(f'Train:       {len(torgo_dataset["train"])}/{original_data_count["train"]} ({len(torgo_dataset["train"]) * 100 // original_data_count["train"]}%)')
        print(f'Validation:  {len(torgo_dataset["validation"])}/{original_data_count["validation"]} ({len(torgo_dataset["validation"]) * 100 // original_data_count["validation"]}%)')
        print(f'Test:        {len(torgo_dataset["test"])}/{original_data_count["test"]} ({len(torgo_dataset["test"]) * 100 // original_data_count["test"]}%)')
        
        print()
        torgo_dataset
    elif pattern == "keep_all":
        pass
    elif pattern == "new":
            # Update the three dataset splits (if ['test_data'] == 1, keep in test, if ['test_data'] == 0, keep in train and validation)
        torgo_dataset['train'] = torgo_dataset['train'].filter(
            lambda x: x['test_data'] == 0)
        torgo_dataset['validation'] = torgo_dataset['validation'].filter(
            lambda x: x['test_data'] == 0)
        torgo_dataset['test'] = torgo_dataset['test'].filter(
            lambda x: x['test_data'] == 1)
    
            # Drop the 'test_data' column
        torgo_dataset['train'] = torgo_dataset['train'].remove_columns([
                                                                       'test_data'])
        torgo_dataset['validation'] = torgo_dataset['validation'].remove_columns([
                                                                                 'test_data'])
        torgo_dataset['test'] = torgo_dataset['test'].remove_columns([
                                                                     'test_data'])

    
    texts = torgo_dataset['train']['text']
    
    cleaned_texts = []
    for text in texts:
        cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
        cleaned_texts.append(cleaned_text)
        
    atypical_speaker_texts[speaker_id] = cleaned_texts


for speaker_id, texts in atypical_speaker_texts.items():
    print(f"{speaker_id}: {len(texts)}")

F01: 9749
F03: 9551
F04: 9551
M01: 9519
M02: 9508
M03: 9493
M04: 9559
M05: 9594


In [11]:
os.makedirs("torgo_lm_arpa_files", exist_ok=True)

for speaker_id, texts in atypical_speaker_texts.items():
    with open(f"torgo_lm_arpa_files/{speaker_id}_texts_{pattern}.txt", "w") as file:
        for text in texts:
            file.write(text + "\n")

In [12]:
arpa_dir = "torgo_lm_arpa_files"
os.makedirs(arpa_dir, exist_ok=True)

for file_name in os.listdir("torgo_lm_arpa_files"):
    if file_name.endswith(f"{pattern}.txt"):
        speaker_id = file_name.split("_")[0]
        arpa_file = f"{arpa_dir}/{speaker_id}_3gram_{pattern}.arpa"
        txt_file = f"torgo_lm_arpa_files/{file_name}"
        !kenlm/build/bin/lmplz -o 3 < "{txt_file}" > "{arpa_file}" -S 10% --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/torgo_lm_arpa_files/F01_texts_new.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 24694 types 1275
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:15300 2:6997409280 3:13120142336
Substituting fallback discounts for order 1: D1=0.5 D2=1 D3+=1.5
Substituting fallback discounts for order 2: D1=0.5 D2=1 D3+=1.5
Statistics:
1 1275 D1=0.808429 D2=1.24331 D3+=1.92209
2 2576 D1=0.5 D2=1 D3+=1.5
3 2134 D1=0.5 D2=1 D3+=1.5
Memory estimate for binary LM:
type     kB
probing 130 assuming -p 1.5
probing 150 assuming -r models -p 1.5
trie     67 without quantization
trie     50 assuming -q 8 -b 8 quantization 
trie     66 assuming -a 22 array pointer compression
trie     48 assuming -a 22 -q 8 -b 8 arra

In [14]:
atypical_speakers = ['F01', 'F03', 'F04', 'M01', 'M02', 'M03', 'M04', 'M05']

for speaker in atypical_speakers:
    arpa_file = f"torgo_lm_arpa_files/{speaker}_3gram_{pattern}.arpa"
    output_file = f"torgo_lm_arpa_files/{speaker}_unigram_{pattern}.txt"

    unigrams = []

    with open(arpa_file, "r") as file:
        start_extraction = False
        for line in file:
            line = line.strip()
            if start_extraction:
                if not line:  
                    break
                parts = line.split("\t")
                unigram = parts[1]  
                unigrams.append(unigram)
            elif line.startswith("\\1-grams:"):
                start_extraction = True

    with open(output_file, "w") as file:
        for unigram in unigrams:
            file.write(unigram + "\n")